<a href="https://colab.research.google.com/github/temple17/dacon_practice/blob/main/1_iris_%EB%AA%A8%EB%B2%94_EDA_%EB%94%B0%EB%9D%BC%ED%95%98%EA%B8%B0_Baseline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
%cd "/content/drive/MyDrive/dacon/1_iris"

/content/drive/MyDrive/dacon/1_iris


In [3]:
iris = pd.read_csv('iris_train.csv')

In [4]:
iris.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2


In [5]:
X = iris[['species', 'sepal length (cm)', 'petal length (cm)']]
X.head()

,species,sepal length (cm),petal length (cm)
0,setosa,4.4,1.4
1,versicolor,6.4,4.5
2,virginica,6.2,4.8
3,virginica,7.2,6.1
4,setosa,4.9,1.4


In [6]:
y_petal = iris['petal width (cm)']
y_sepal = iris['sepal width (cm)']

In [7]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [8]:
iris_cat = iris[['species']]

In [9]:
iris_cat_encoded = ordinal_encoder.fit_transform(iris_cat)

In [10]:
ordinal_encoder.categories_

[array(['setosa', 'versicolor', 'virginica'], dtype=object)]

In [14]:
X['species'] = iris_cat_encoded

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
X.head()

,species,sepal length (cm),petal length (cm)
0,0.0,4.4,1.4
1,1.0,6.4,4.5
2,2.0,6.2,4.8
3,2.0,7.2,6.1
4,0.0,4.9,1.4


In [16]:
X = np.array(X)

In [17]:
# y = w1x1 + w2x2 + w3x3 + w4 
# x1 ~ x3 : species, sepal length, petal length

In [18]:
ones = np.ones_like(X[:,0]) # 1로만 되어있는 (75,) 벡터 생성
ones = ones.reshape(75,-1)
X = np.concatenate((X,ones), axis = 1)

X[:10]

array([[0. , 4.4, 1.4, 1. ],
       [1. , 6.4, 4.5, 1. ],
       [2. , 6.2, 4.8, 1. ],
       [2. , 7.2, 6.1, 1. ],
       [0. , 4.9, 1.4, 1. ],
       [2. , 6.5, 5.8, 1. ],
       [0. , 4.3, 1.1, 1. ],
       [1. , 6.7, 5. , 1. ],
       [1. , 6.8, 4.8, 1. ],
       [1. , 6.6, 4.4, 1. ]])

In [19]:
transpose_doted_X = X.T.dot(X)
inversed = np.linalg.inv(transpose_doted_X)
doted_inv_t = inversed.dot(X.T)
weight_petal = doted_inv_t.dot(y_petal)
weight_petal

array([ 0.29159172, -0.10650362,  0.33433041,  0.25399605])

In [20]:
prediction_petal = X.dot(weight_petal)
error_petal = sum(abs(prediction_petal - y_petal))/len(prediction_petal)
error_petal

0.13392975138266672

In [21]:
weight_sepal = doted_inv_t.dot(y_sepal)
weight_sepal

array([ 0.19126075,  0.54550355, -0.39424673,  1.11109575])

In [22]:
prediction_sepal = X.dot(weight_sepal)
error_sepal = sum(abs(prediction_sepal - y_sepal))/len(prediction_sepal)
error_sepal

0.2402450093944724

In [24]:
from sklearn.linear_model import LinearRegression

model_petal = LinearRegression()
model_petal.fit(X[:,:3], y_petal)

model_sepal = LinearRegression()
model_sepal.fit(X[:,:3], y_sepal)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [25]:
prediction_sepal_scikit = model_sepal.predict(X[:,:3])
scikit_sepal_error = sum(abs(prediction_sepal_scikit - y_sepal))/len(prediction_sepal_scikit)
scikit_sepal_error

0.24024500939447221

In [26]:
prediction_petal_scikit = model_petal.predict(X[:,:3])
scikit_petal_error = sum(abs(prediction_petal_scikit - y_petal))/len(prediction_petal_scikit)
scikit_petal_error

0.13392975138266616

In [27]:
print("직접 구현한 Normal equation 꽃잎 너비 error:          ", error_petal)
print("sklearn으로 LinearRegression모델을 사용한 꽃잎 너비 error:", scikit_petal_error)

print("직접 구현한 Normal equation 꽃받침 너비 error:          ", error_sepal)
print("sklearn으로 LinearRegression모델을 사용한 꽃받침 너비 error:", scikit_sepal_error)

직접 구현한 Normal equation 꽃잎 너비 error:           0.13392975138266672
sklearn으로 LinearRegression모델을 사용한 꽃잎 너비 error: 0.13392975138266616
직접 구현한 Normal equation 꽃받침 너비 error:           0.2402450093944724
sklearn으로 LinearRegression모델을 사용한 꽃받침 너비 error: 0.24024500939447221


In [29]:
submission = pd.read_csv('sample_submission.csv')
test_set = pd.read_csv('iris_test.csv')

In [31]:
test_cat = test_set[['species']]

test_cat_encoded = ordinal_encoder.fit_transform(test_cat)

In [32]:
test_set['species'] = test_cat_encoded

In [34]:
X = test_set[['species', 'sepal length (cm)', 'petal length (cm)']]

In [35]:
pred_petal = model_petal.predict(X)

In [36]:
pred_sepal = model_sepal.predict(X)

In [37]:
submission['sepal width (cm)'] = pred_sepal
submission['petal width (cm)'] = pred_petal

submission.to_csv('submission2.csv', index = False)